In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch


In [2]:
kenh = {
    "Đời sống":
    ["https://kenh14.vn/xem-an-choi.chn",
     "https://kenh14.vn/doi-song/nhan-vat.chn",
     "https://kenh14.vn/doi-song/tram-yeu.chn",
     "https://kenh14.vn/money-z.chn", ],
    "Thời sự":
    ["https://kenh14.vn/xa-hoi/song-xanh.chn",
     "https://kenh14.vn/xa-hoi/phap-luat.chn",
     "https://kenh14.vn/xa-hoi/nong-tren-mang.chn", ],
    "Quốc tế":
    ["https://kenh14.vn/the-gioi/di.chn",
     "https://kenh14.vn/the-gioi/chum-anh.chn",
     "https://kenh14.vn/the-gioi/kham-pha.chn", ],
    "Sao":
    ["https://kenh14.vn/star/sao-viet.chn",
     "https://kenh14.vn/star/hoi-ban-than-showbiz.chn",
     "https://kenh14.vn/star.chn", ],
    "Điện ảnh":
    ["https://kenh14.vn/tv-show.chn",
     "https://kenh14.vn/cine.chn",
     "https://kenh14.vn/cine/phim-chieu-rap.chn",
     "https://kenh14.vn/cine/phim-viet-nam.chn",
     "https://kenh14.vn/cine/series-truyen-hinh.chn",
     "https://kenh14.vn/cine/hoa-ngu-han-quoc.chn", ],
    "Âm nhạc":
    ["https://kenh14.vn/musik.chn",
     "https://kenh14.vn/musik/au-my.chn",
     "https://kenh14.vn/musik/chau-a.chn",
     "https://kenh14.vn/musik/viet-nam.chn",
     "https://kenh14.vn/musik/hip-hop-neva-die.chn", ],
    "Làm đẹp":
    ["https://kenh14.vn/beauty-fashion.chn",
     "https://kenh14.vn/beauty-fashion/star-style.chn",
     "https://kenh14.vn/beauty-fashion/lam-dep.chn",
     "https://kenh14.vn/beauty-fashion/thoi-trang.chn", ],
    "Mua sắm":
    ["https://kenh14.vn/xem-mua-luon.chn",
     "https://kenh14.vn/xem-mua-luon/thoi-trang.chn",
     "https://kenh14.vn/xem-mua-luon/dep.chn",
     "https://kenh14.vn/xem-mua-luon/house-n-home.chn", ],
    "Giải trí":
    ["https://kenh14.vn/an-quay-di.chn",
     "https://kenh14.vn/an-quay-di/an.chn",
     "https://kenh14.vn/an-quay-di/quay.chn",
     "https://kenh14.vn/an-quay-di/di.chn", ],
    "Thể thao":
    ["https://kenh14.vn/sport.chn",
     "https://kenh14.vn/sport/bong-da.chn",
     "https://kenh14.vn/sport/hau-truong.chn", ],
    "Học đường":
    ["https://kenh14.vn/hoc-duong.chn",
     "https://kenh14.vn/hoc-duong/tuyen-sinh.chn",
     "https://kenh14.vn/hoc-duong/nguoi-viet-tre.chn",
     "https://kenh14.vn/hoc-duong/du-hoc.chn",
     "https://kenh14.vn/hoc-duong/ban-tin-46.chn", ],
    "Thể thao điện tử":
    ["https://kenh14.vn/esports.chn",
     "https://kenh14.vn/esports/cao-thu.chn",
     "https://kenh14.vn/esports/dau-truong.chn",
     "https://kenh14.vn/esports/dau-truong-danh-vong.chn", ],
    "Sức khỏe":
    ["https://kenh14.vn/suc-khoe.chn",
     "https://kenh14.vn/suc-khoe/kien-thuc-gioi-tinh.chn", ],
    "Công nghệ":
    ["https://kenh14.vn/2-tek/mobile.chn",
     "https://kenh14.vn/2-tek/concept.chn",
     "https://kenh14.vn/2-tek/ung-dung-thu-thuat.chn",
     "https://kenh14.vn/2-tek/cong-nghe-vui.chn",
     "https://kenh14.vn/2-tek.chn", ]
}


In [3]:
kenhLinks = {}
for key in kenh:
    kenhLinks[key] = []
for key, value in kenh.items():
    for url in value:
        try:
            response = requests.get(url)
            tree = lxml.html.fromstring(response.content)
            title_elem = tree.cssselect('.knswli-title a')
            for elem in title_elem:
                kenhLinks[key].append("https://kenh14.vn"+elem.get('href'))
        except:
            pass

for key in kenhLinks:
    print(key, len(kenhLinks[key]))

Đời sống 21
Thời sự 23
Quốc tế 14
Sao 21
Điện ảnh 44
Âm nhạc 34
Làm đẹp 40
Mua sắm 20
Giải trí 32
Thể thao 23
Học đường 40
Thể thao điện tử 29
Sức khỏe 14
Công nghệ 37


In [4]:
kenhContent = {}
for key in kenh:
    kenhContent[key] = []
for key, value in kenhLinks.items():
    for url in value:
        if(url.startswith('https://kenh14.vn/')):
            try:
                id = url.split('-')[-1].replace('.chn', '')
                downloaded = trafilatura.fetch_url(url)
                tree = lxml.html.fromstring(str(downloaded))
                date_elem = len(tree.cssselect(
                    ".kbwcm-time")) and tree.cssselect(".kbwcm-time")[0].text or ""
                content = trafilatura.extract(downloaded)
                content = content.replace('\n', ' ')
                item = {"domain": "https://kenh14.vn/", "URL": url,
                        "content": content, "date": date_elem, "id": id}
                kenhContent[key].append(item)
            except:
                pass

for key in kenhContent:
    print(key, len(kenhContent[key]))


ERROR:root:pycurl: https://kenh14.vn/mach-ban-meo-chong-toc-xoan-roi-20220516160350434.chn (28, 'Failed to connect to kenh14.vn port 443 after 21033 ms: Timed out')
ERROR:root:pycurl: https://kenh14.vn/10-kieu-trang-phuc-khien-voc-dang-manh-mai-cung-thanh-nang-ne-to-ngang-20220514074757858.chn (28, 'Failed to connect to kenh14.vn port 443 after 21027 ms: Timed out')
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None
ERROR:trafilatura.core:not enough text None


Đời sống 21
Thời sự 23
Quốc tế 14
Sao 21
Điện ảnh 44
Âm nhạc 34
Làm đẹp 38
Mua sắm 20
Giải trí 32
Thể thao 23
Học đường 40
Thể thao điện tử 29
Sức khỏe 14
Công nghệ 37


In [5]:
client = Elasticsearch("http://localhost:9200")

client.info()


/home/hplong/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'HoangPhiLong-20182651',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'jF2zfdDdQqueQik2u6h9Ug',
 'version': {'number': '7.17.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': 'de7261de50d90919ae53b0eff9413fd7e5307301',
  'build_date': '2022-03-28T15:12:21.446567561Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
def gendata():
    for key,value in kenhContent.items():
         for item in value:
            temp_item = {k: item[k] for k in set(list(item.keys())) - set(['id'])}
            temp_item['category'] = key
            client.index(index="journal", id=item['id'], document=temp_item)

gendata()